In [1]:
import geopandas as gpd
import sqlite3
import pandas as pd
from shapely import wkt
import pyogrio
from tqdm import tqdm
import numpy as np

geopackage_path = "C:/Users/eleonore.kong/Documents/InSAR/DATA/lpf_egms_data.gpkg"
communes_path = "C:/Users/eleonore.kong/Documents/InSAR/communes_catnat_secheresse.gpkg"
output = "C:/Users/eleonore.kong/Documents/InSAR/DATA/"

gdf_communes = pyogrio.read_dataframe(communes_path, layer='communes')
gdf_communes = gdf_communes[['id', 'nom_m', 'insee_com', 'geometry']]
gdf_communes = gdf_communes.to_crs("EPSG:2154")
gdf_insar = pyogrio.read_dataframe(geopackage_path, layer='data_rga')

In [2]:
percentile_75 = lambda x: x.quantile(0.75)

In [3]:
i=0
chunk_size=1000
ids = gdf_communes['id'].unique()
ids_sublists = [ids[i:i + chunk_size] for i in range(0, len(ids), chunk_size)]
median_gdf = {}
geom = {}

for ids_list in tqdm(ids_sublists):
    com = gdf_communes[gdf_communes['id'].isin(ids_list)]
    
    gdf = com.sjoin(gdf_insar, how='left')
    gdf = gdf[(abs(gdf['gap_insar_swi_trough_days']) <= 30) | (abs(gdf['gap_insar_swi_peak_days']) <= 30)]
    
    if not gdf.empty :
        median_df = gdf.groupby(['id', 'year', 'Type']).agg({ 
            col : ['median'] for col in gdf.select_dtypes(include='number').columns #, percentile_75
        })
        median_df.columns = ['_'.join(col).strip() for col in median_df.columns]
        # median_df.columns = [col.replace('<lambda_0>', 'p75') for col in median_df.columns]

        median_df = median_df.reset_index()
        median_df = median_df.merge(gdf_communes, left_on='id', right_on='id', how='left')
        median_df.drop('id', axis=1, inplace=True)
        median_df = gpd.GeoDataFrame(median_df, crs=2154)

        # column_name_diff_per_year = 'summer_winter_amplitude_diff_'

        # for stat in ('median', 'p75'):
        #     median_df[stat + '_summer_winter_amplitude_diff'] = median_df.apply(lambda row : med_summer_winter_amplitude_diff(row, stat), axis=1)
        
            # median_df[stat + '_evolution_amplitude'] = median_df.apply(lambda row : amplitude_evolution(row, stat), axis=1)
        
            # for season in ['summer', 'winter']:
            #     median_df[stat + '_' + season + '_amplitude'] = median_df.apply(lambda row : med_seasonal_amplitude(row, season, stat), axis=1)
            #     median_df[stat + '_evolution_'+ season +'_amplitude'] = median_df.apply(lambda row : seasonal_amplitude_evolution(row, season, stat), axis=1)
        
        if i > 0:
            pyogrio.write_dataframe(median_df, output + 'grid_stats_agg_lpf.gpkg', layer='communes', driver="GPKG", append=True)
        else:
            pyogrio.write_dataframe(median_df, output + 'grid_stats_agg_lpf.gpkg', layer='communes', driver="GPKG")

    i += 1

100%|██████████| 35/35 [20:08<00:00, 34.52s/it]
